In [ ]:
import os
import glob
import csv
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import shutil

In [ ]:
# 시각화 함수
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(124)]

def draw_box(box, image, color, class_number=None):
    x1, x2, y1, y2 = [int(c) for c in box]
    thickness = round(0.002 * (image.shape[0] + image.shape[1]) / 2) + 1
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness=thickness, lineType=cv2.LINE_AA)
    if class_number is not None:
        cv2.putText(image, str(class_number), (int(x1*0.98), int(y1*0.98)), cv2.FONT_ITALIC, 2, color, 2, cv2.LINE_AA)

        
def draw_bboxes(image, labels, colors=None):
    """
    image : 이미지
    labels : label [클래스, min_x, min_y, width, height] 들이 담겨있는 리스트
    """
    if colors is None:
        colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(124)]
    height, width, channels = image.shape
    for cl, x, y, w, h in labels:
        cl = int(cl)
        x = float(x) * width
        y = float(y) * height
        w = float(w) * width
        h = float(h) * height
        x1, x2 = round(x - w / 2), round(x + w / 2)
        y1, y2 = round(y - h / 2), round(y + h / 2)
        draw_box([x1, x2, y1, y2], image, colors[cl], cl)

In [ ]:
def get_label(path):
    with open(path) as f:
        r = csv.reader(f, delimiter=' ')
        label_list = list(r)
    return label_list

def get_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_img_label(img_path, label_path):
    img = get_img(img_path)
    label = get_label(label_path)
    return img, label

In [ ]:
# image, label list
path='./final-project-level2-cv-05/yolov7/dataset/'
img_path = path + 'images/'
label_path = path + 'labels/'
img_list = sorted(os.listdir(img_path))
label_list = sorted(os.listdir(label_path))

In [ ]:
# sessions 리스트에 session 단위로
sessions = []
prefixes = [str(i) + '_' for i in range(1, 13)] + ['3rd Run_']
for prefix in prefixes:
    session = [(label[:-4] + '.jpg', label) for label in label_list if label.startswith(prefix)]
    sessions.append(session)
len(sessions)

In [ ]:
def get_box_coord(label, width, height):
    """
    x1 : min_x
    x2 : max_x
    y1 : min_y
    y2 : max_y
    """
    cl, x, y, w, h = label
    cl = int(cl)
    x = float(x) * width
    y = float(y) * height
    w = float(w) * width
    h = float(h) * height
    x1, x2 = round(x - w / 2), round(x + w / 2)
    y1, y2 = round(y - h / 2), round(y + h / 2)
    return x1, x2, y1, y2

def box_coord_to_yolo_label(cl, box, width, height):
    x1, x2, y1, y2 = box
    x = (x1 + x2) / 2 / width
    y = (y1 + y2) / 2 / height
    w = (x2 - x1) / width
    h = (y2 - y1) / height
    return [cl, x, y, w, h]

In [ ]:
def crop_image(img, box):
    x1, x2, y1, y2 = box
    cropped_img = img[y1:y2, x1:x2]
    return cropped_img

In [ ]:
def resize(src, src_area, dst_area, dst_w, dst_h):
    if src_area > dst_area:
        resized = cv2.resize(src, (dst_w, dst_h), interpolation=cv2.INTER_AREA)
    elif src_area < dst_area:
        resized = cv2.resize(src, (dst_w, dst_h), interpolation=cv2.INTER_CUBIC)
    else:
        resized = src
    return resized


def resize_paste(src, src_area, dst, dst_area, dst_w, dst_h, dst_box):
    resized = resize(src, src_area, dst_area, dst_w, dst_h)
    dst[dst_box[2]:dst_box[3], dst_box[0]:dst_box[1]] = resized


def zero_padding(src, src_w, src_h, dst_w, dst_h):
    shift_x = (dst_w - src_w) / 2
    shift_y = (dst_h - src_h) / 2
    m = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    dsize = (dst_w, dst_h)
    padded = cv2.warpAffine(src, m, dsize)
    return padded


def padding_paste(src, src_w, src_h, dst, dst_w, dst_h, dst_box):
    padded = zero_padding(src, src_w, src_h, dst_w, dst_h)
    dst[dst_box[2]:dst_box[3], dst_box[0]:dst_box[1]] = padded


def common_paste(src, src_area, src_w, src_h, dst, dst_area, dst_w, dst_h, dst_box):
    if src_area > dst_area:
        w1 = (src_w - dst_w) // 2
        w2 = src_w - dst_w - w1
        h1 = (src_h - dst_h) // 2
        h2 = src_h - dst_h - h1
        x1, x2 = dst_box[0] - w1, dst_box[1] + w2
        y1, y2 = dst_box[2] - h1, dst_box[3] + h1
    elif src_area < dst_area:
        w1 = (dst_w - src_w) // 2
        w2 = dst_w - src_w - w1
        h1 = (dst_h - src_h) // 2
        h2 = dst_h - src_h - h1
        x1, x2 = dst_box[0] + w1, dst_box[1] - w2
        y1, y2 = dst_box[2] + h1, dst_box[3] - h2
    else:
        x1, x2, y1, y2 = dst_box
    dst[y1:y2, x1:x2] = src
    return [x1, x2, y1, y2]

## crop_paste 적용 알고리즘

In [ ]:
# image, label list
path='./bootcamp/dataset/'
img_path = path + 'images/'
label_path = './bootcamp/dataset/labels124/'
img_list = sorted(os.listdir(img_path))
label_file_list = sorted(os.listdir(label_path))

In [ ]:
# img_list(약 25000개) 정의
with open('pkt_train_124.txt', 'r') as f:
    img_list = f.readlines()
    for i, img in enumerate(img_list):
        img_list[i] = img_list[i].strip()
        img_list[i] = img_list[i].split('/')[2]

In [ ]:
from tqdm import tqdm
# label_list(약 25000개) 정의
label_list_temp1 = img_list.copy()
for i, label in enumerate(label_list_temp1):
    label_list_temp1[i] = label.replace('.jpg', '.txt')
label_list_temp2 = label_list_temp1.copy()
for i, label in tqdm(enumerate(label_list_temp1)): # txt 파일 존재하지 않는 경우 예외 처리
    if label not in label_file_list:
        label_list_temp2.remove(label)

In [ ]:
label_list = label_list_temp2.copy()
for i, label in tqdm(enumerate(label_list_temp2)): # txt 파일이 비어 있는 경우 예외 처리
    lb=get_label(label_path+label)
    if len(lb)==0:
        label_list.remove(label)
len(label_list)

In [ ]:
# sessions 리스트에 session 단위로
sessions = []
prefixes = [str(i) + '_' for i in range(1, 13)] + ['3rd Run_']
for prefix in prefixes:
    session = [(label[:-4] + '.jpg', label) for label in label_list if label.startswith(prefix)]
    sessions.append(session)
len(sessions)

In [ ]:
def crop_image(img, box, center_crop=False):
    x1, x2, y1, y2 = box
    if center_crop:
        # center crop 시, 기존 w, h의 84%에 해당하는 길이 즉, 0.84*0.84=70% center 영역 crop
        center_x, center_y = int((x1+x2)/2), int((y1+y2)/2)
        w, h = x2-x1, y2-y1
        center_crop_w, center_crop_h = 0.84*w, 0.84*h 
        center_crop_w_2, center_crop_h_2 = int(center_crop_w/2), int(center_crop_h/2)
        cropped_img = img[center_y-center_crop_h_2:center_y+center_crop_h_2, center_x-center_crop_w_2:center_x+center_crop_w_2]
        if len(cropped_img)==0:  # ceter crop 된 영역이 너무 작아서, 빈 리스트인 경우 center crop x -> 일반 crop
            cropped_img = img[y1:y2, x1:x2]
    else:
        cropped_img = img[y1:y2, x1:x2]
        
    return cropped_img

def resize(src, src_area, dst_area, dst_w, dst_h):
    if src_area == 0 or dst_area ==0:  # bbox 매우 작아서 area 0이면 pass
        return None
    if src_area > dst_area:
        resized = cv2.resize(src, (dst_w, dst_h), interpolation=cv2.INTER_AREA)
    else:
        resized = cv2.resize(src, (dst_w, dst_h), interpolation=cv2.INTER_CUBIC)
    # elif src_area < dst_area:
    #     resized = cv2.resize(src, (dst_w, dst_h), interpolation=cv2.INTER_CUBIC)
    # else:
    #     resized = src
        
    return resized

def resize_paste(src, src_area, dst, dst_area, dst_w, dst_h, dst_box):
    resized = resize(src, src_area, dst_area, dst_w, dst_h)
    if resized is None:
        return None
    pasted_image = dst.copy()
    pasted_image[dst_box[2]:dst_box[3], dst_box[0]:dst_box[1]] = resized
    
    return pasted_image
    
def crop_paste(src_img, src_box, dst_img, dst_box, paste_method='resize', center_crop=False):
    """
    src_img의 box를 crop해서 dst_img의 box 위치에 paste
    
    src_img : crop할 image
    src_box : crop할 src_img의 bbox
    dst_img : paste할 image
    dst_box : paste할 dst_img의 bbox
    paste_method : 'resize', 'padding', 'paste'
        'resize' : crop된 img1의 box를 img2의 box와 동일한 크기로 resize
        'padding' : img2의 box가 더 클 때 남는 부위를 zero padding, 작을 때는 'paste'와 동일
        'paste' : 그냥 붙여넣는다.
    center_crop : 기존 box 영역의 약 70%에 해당하는 center 영역을 crop, default:False
    """
    if paste_method not in ['resize']:
        raise ValueError("paste_method : 'resize'")

    
    # return할 label 값. crop할 박스의 class + paste될 box
    ret_box = src_box[:1] + dst_box[1:]
    
    # 각 이미지의 height, width 값
    src_img_height, src_img_width, _ = src_img.shape
    dst_img_height, dst_img_width, _ = dst_img.shape
    
    # resize 
    src_img = resize(src_img, src_img_height*src_img_width, dst_img_height*dst_img_width, dst_img_width, dst_img_height)
    src_img_height, src_img_width, _ = src_img.shape
    
    # min_x, max_x, min_y, max_y 값으로 변환
    src_box = get_box_coord(src_box, src_img_width, src_img_height)
    dst_box = get_box_coord(dst_box, dst_img_width, dst_img_height)

    # box의 width와 height 값
    src_box_w, src_box_h = src_box[1] - src_box[0], src_box[3] - src_box[2]
    dst_box_w, dst_box_h = dst_box[1] - dst_box[0], dst_box[3] - dst_box[2]
    
    # box의 크기 
    src_box_area = src_box_w * src_box_h
    dst_box_area = dst_box_w * dst_box_h
  
    # image crop
    cropped_img = crop_image(src_img, src_box, center_crop)
    
    if paste_method == 'resize':
        pasted_image = resize_paste(cropped_img, src_box_area, dst_img, dst_box_area, dst_box_w, dst_box_h, dst_box)
    # elif paste_method == 'padding' and src_box_area < dst_box_area:
    #     padding_paste(cropped_img, src_box_w, src_box_h, dst_img, dst_box_w, dst_box_h, dst_box)
    # else:
    #     modified_box = common_paste(cropped_img, src_box_area, src_box_w, src_box_h, 
    #                                 dst_img, dst_box_area, dst_box_w, dst_box_h, dst_box)
    #     ret_box = box_coord_to_yolo_label(ret_box[0], modified_box, dst_img_width, dst_img_height)
    return pasted_image, ret_box

In [ ]:
from itertools import combinations

def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    
    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

def calculate_iou_in_list(bbox_list):
    """
    input: [[cls, x, y, w, h], [cls, x, y, w, h] ...] 
    
    output: {box_idx:[max iou], ...}
    """
    x1y1x2y2_box_list = []
    for bbox in bbox_list:
        x1, x2, y1, y2 = get_box_coord(bbox, width=1920, height=1080)
        x1y1x2y2_box = [x1, y1, x2, y2]
        x1y1x2y2_box_list.append(x1y1x2y2_box)
    
    iou_dict = {idx:[] for idx in range(len(x1y1x2y2_box_list))}
    box_combination = list(combinations(list(range(len(x1y1x2y2_box_list))), 2))
    
    for combi in box_combination:
        bbox1, bbox2 = x1y1x2y2_box_list[combi[0]], x1y1x2y2_box_list[combi[1]]
        iou = IoU(bbox1, bbox2)
        iou_dict[combi[0]].append(iou)
        iou_dict[combi[1]].append(iou)
    
    for key in iou_dict.keys():
        iou_dict[key] = max(iou_dict[key])
        
    return iou_dict

In [ ]:
# crop paste test, cetercrop - non-centercrop compare
tup1 = sessions[0][10]
tup2 = sessions[1][15]
im1, lb1 = get_img_label(img_path+tup1[0], label_path+tup1[1])
im2, lb2 = get_img_label(img_path+tup2[0], label_path+tup2[1])
pasted_image, ret_box=crop_paste(im2, lb2[0], im1, lb1[1], paste_method='resize', center_crop=False)
pasted_image2, ret_box2=crop_paste(im2, lb2[0], im1, lb1[1], paste_method='resize', center_crop=True)

fig = plt.figure(figsize=(40, 40))
ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(pasted_image)
ax1.set_xlabel('no center crop')
ax1.set_xticks([]), ax1.set_yticks([])
 
ax2 = fig.add_subplot(1, 2, 2)
ax2.imshow(pasted_image2)
ax2.set_xlabel('center crop')
ax2.set_xticks([]), ax2.set_yticks([])
 
plt.show()

In [ ]:
img_path = './final-project-level2-cv-05/yolov7/dataset/images/'
label_path = './final-project-level2-cv-05/yolov7/dataset/labels/'
save_img_label_path = './final-project-level2-cv-05/yolov7/dataset/ratio_base_center_crop_pasted_images/' 
select_by_box_size = False

for idx, session in tqdm(enumerate(sessions)):
    session_candidate_list = list(range(len(sessions)))
    session_candidate_list.remove(idx) # session 랜덤 선택 위해, 현재 session 제외
    # base_image, selected_box, src_image, src_box
    for img, lb in session:
        base_image, base_label = get_img_label(img_path+img, label_path+lb)
        
        # img(base) 내 bbox 랜덤 선택
        selected_box_idx = random.choice(list(range(len(base_label))))
        selected_box = base_label[selected_box_idx]
        
        # session 랜덤 선택 
        selected_session_idx = random.choice(session_candidate_list)
        
        # session 내 img(crop source) 랜덤 선택
        src_img_name, src_label_name = random.choice(sessions[selected_session_idx])

        # source img 내 bbox 선택 -> bbox 1개인 경우 해당 bbox 선택
        src_image, src_label = get_img_label(img_path+src_img_name, label_path+src_label_name)
        if len(src_label) == 1:
            src_box = src_label[0]
            
        # bbox 여러개 인 경우 -> 1. IoU 기반 후보 선정, 2. selected_box의 area와 가장 비슷한 area 가지는 bbox 선정
        #                                           or selected_box의 aspect ratio와 가장 비슷한 ratio 가지는 bbox 선정
        else:       
            iou_dict = calculate_iou_in_list(src_label)
            iou_dict_values = list(iou_dict.values())
            min_iou = min(iou_dict_values)
            if min_iou > 0.5: # min iou가 0.5보다 크면 원본 저장하고 pass
                base_image = cv2.cvtColor(base_image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(save_img_label_path+img, base_image)
                with open(save_img_label_path+lb, 'w') as f:
                    for box in base_label:
                        row = ' '.join(box)
                        f.write(f"{row}\n")
                continue
                
            # 최소가 한개인 경우, 해당 bbox 선택 
            if iou_dict_values.count(min_iou) == 1:
                src_box_idx = iou_dict_values.index(min_iou)
                src_box = src_label[src_box_idx]
            # 최소가 여러개 일 경우, base_box의 size와 가장 비슷한 size를 가지는 bbox 선택 
            #                    or base_box의 aspect_ratio와 가장 비슷한 aspect_ratio를 가지는 bbox 선택 
            else:
                # box size 기준 
                if select_by_box_size:
                    base_box_area = float(selected_box[3]) * float(selected_box[4])
                    min_iou_indices_list = [idx for idx, iou in enumerate(iou_dict_values) if iou==min_iou]
                    src_box_area_list = [float(src_label[idx][3])*float(src_label[idx][4]) for idx in min_iou_indices_list]
                    area_gap_list = [abs(base_box_area-area) for area in src_box_area_list]
                    src_box_idx_idx = area_gap_list.index(min(area_gap_list))
                    src_box_idx = min_iou_indices_list[src_box_idx_idx]
                    src_box = src_label[src_box_idx]
                # box aspect ratio(h/w) 기준
                else:
                    base_box_aspect_ratio = float(selected_box[4]) / float(selected_box[3]) 
                    min_iou_indices_list = [idx for idx, iou in enumerate(iou_dict_values) if iou==min_iou]
                    src_box_aspect_ratio_list = [float(src_label[idx][4])/float(src_label[idx][3]) for idx in min_iou_indices_list]
                    aspect_ratio_gap_list = [abs(base_box_aspect_ratio-aspect_ratio) for aspect_ratio in src_box_aspect_ratio_list]
                    src_box_idx_idx = aspect_ratio_gap_list.index(min(aspect_ratio_gap_list))
                    src_box_idx = min_iou_indices_list[src_box_idx_idx]
                    src_box = src_label[src_box_idx]
                    
                
        pasted_image, ret_box = crop_paste(src_image, src_box, base_image, selected_box, paste_method='resize', center_crop=True)
        
        if pasted_image is None:
            base_image = cv2.cvtColor(base_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(save_img_label_path+img, base_image)
            with open(save_img_label_path+lb, 'w') as f:
                for box in base_label:
                    row = ' '.join(box)
                    f.write(f"{row}\n")
            continue
        
        base_label[selected_box_idx] = ret_box
        new_image = cv2.cvtColor(pasted_image, cv2.COLOR_RGB2BGR)
        new_label = base_label
        
        # img, txt 파일 저장
        cv2.imwrite(save_img_label_path+img, new_image)
        with open(save_img_label_path+lb, 'w') as f:
            for box in new_label:
                row = ' '.join(box)
                f.write(f"{row}\n")
                
    #     print(base_box_aspect_ratio)
    #     print(min_iou_indices_list)
    #     print(src_box_aspect_ratio_list)
    #     print(aspect_ratio_gap_list)
    #     print(src_box_idx_idx, src_box_idx)
    #     break
    # break

In [ ]:
### txt 파일 생성

In [ ]:
crop_pasted_image_path = 'final-project-level2-cv-05/yolov7/dataset/ratio_base_center_crop_pasted_images'
crop_pasted_img_lb_list = os.listdir(crop_pasted_image_path)

In [ ]:
crop_pasted_img_list = []
for file in crop_pasted_img_lb_list:
    if file[-1] == 'g':
        crop_pasted_img_list.append(file)
len(crop_pasted_img_list)

In [ ]:
no_label_img_list = [] # label이 존재하지 않는 image list 
train_img_list = os.listdir('final-project-level2-cv-05/yolov7/dataset/images')
yes_label_img_list = os.listdir('final-project-level2-cv-05/yolov7/dataset/labels')
for i in tqdm(range(len(train_img_list))):
    img_path = train_img_list[i]
    img_path_split = img_path.split('.')
    img_name = img_path_split[0] + '.' + img_path_split[1] + '.txt'
    if img_name not in yes_label_img_list:
        no_label_img_list.append(img_path)
len(no_label_img_list) == 81162 - 73566

In [ ]:
## 라벨이 존재하지 않는 파일 추가

In [ ]:
import random
random.seed(41)
random.shuffle(no_label_img_list)

In [ ]:
with open('final-project-level2-cv-05/yolov7/dataset/ratio_base_center_crop_pasted_pkt_train_124.txt', 'w') as f:
    for img in crop_pasted_img_list:
        img_path = 'dataset/ratio_base_center_crop_pasted_images/' + img
        f.write(f"{img_path}\n")
    for j in range(2585):
        no_label_img_path = 'dataset/ratio_base_center_crop_pasted_images/' + no_label_img_list[j] + '\n'
        f.write(no_label_img_path)

In [ ]:
for i,img in enumerate(no_label_img_list[:2585]):
    img_path = os.path.join('final-project-level2-cv-05/yolov7/dataset/images',img)
    shutil.copyfile(img_path, 'final-project-level2-cv-05/yolov7/dataset/ratio_base_center_crop_pasted_images/'+img)

In [ ]:
len(os.listdir('final-project-level2-cv-05/yolov7/dataset/ratio_base_center_crop_pasted_images'))

### crop paste 이미지 확인

In [ ]:
crop_paste_img_list = [i for i in os.listdir('./final-project-level2-cv-05/yolov7/dataset/ratio_base_crop_pasted_images') if i.endswith('jpg')]
crop_pasted_img_label_path = './final-project-level2-cv-05/yolov7/dataset/ratio_base_crop_pasted_images/'

In [ ]:
example_img = crop_paste_img_list[3034] # change index, 3032 3034 3035

In [ ]:
im1, lb1 = get_img_label(crop_pasted_img_label_path+example_img, crop_pasted_img_label_path+example_img.replace('jpg', 'txt'))
draw_bboxes(im1, lb1, colors=colors)
plt.figure(figsize=(20, 30))
plt.imshow(im1)